# Using Function Calls in ChatGPT API


## Getting Started

To get started, create an Open AI API account, set up billing, and generate and API key at https://platform.openai.com/. Create a file called `.env`, and add a line `OPENAI_API_KEY=<your-openai-api-key>`. This key will be read by the `load_dotenv` library.

Install the `openai` Python package with `pip -r requirements.txt`, and run the code below to read the key.

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

# Load key from an environmental variable called "OPENAI_API_KEY"
# Use python-dotenv https://pypi.org/project/python-dotenv/
# And take environment variables from .env
client = OpenAI()

## Making Function Calls

ChatGPT does not have access to real-time data such as the weather, or proprietary information from your company that you may want to access. You can use the power of ChatGPT to understand a request from the user, and call functions that you define.

Let us say you have defined a function to get the weather. (In this case it returns hard-coded information.) The function can take parameters.

In [ ]:
import json


def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

If you describe this function and it's parameters to ChatGPT, ChatGPT will determine the intent to use this function.

In [ ]:
user_prompt = "What's the weather like in Boston?"

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": user_prompt}],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Both the city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit for temperature",
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call="auto",
)

We name and describe the function, then describe the parameters that to pass to this function. ChatGPT relies on this description to help identify whether to call the function, and what the arguments should be. 

If we specify `function_call="auto"`, ChatGPT will try to fulfill the function parameters. If set to `none` you can force no function to be detected. 


Here is the prompt returned:

In [ ]:
reply_content = chat_completion.choices[0].message
reply_content

Notice that there is no content, but instead a function call, with extracted parameters in a JSON object. It is your responsibility to call the function if you would like to do that.

## References

- Based on https://github.com/Sentdex/ChatGPT-API-Basics
- See [ChatGPT API in Python by sentdex](https://www.youtube.com/watch?v=c-g6epk3fFE)